# Creating the Network
GANs consist of two parts: 1. a generator, G, mapping from an input space (with elements here named z) to an output space, and 2. a discriminator, D, which attempts to distinguish between points in a true underlying distribution and those generated by G.

Below is a simple GAN of (arbitrarily many) fully connected layers for each of G and D. The input and outputs can be any dimensionality, and the true data distribution can be defined in any manner (note however that the plotting will only work nicely for 2D output spaces).

In [ ]:
from keras.layers import Input, Dense, Convolution2D, Conv2DTranspose, MaxPooling2D, UpSampling2D, Add
from keras.models import Model
from keras.models import load_model
from keras.utils import plot_model
from keras.callbacks import TensorBoard, History
from keras import backend as K
import tensorflow as tf 
from keras.backend.tensorflow_backend import set_session

In [ ]:
import numpy as np
import h5py 
import matplotlib.pyplot as plt
import os

In [ ]:
os.chdir('../data')
#print(os.getcwd())

f = h5py.File('X_32_n.h5','r') 
X = f.get('X') 
X = np.array(X)

f = h5py.File('Y_32_n.h5','r') 
Y = f.get('Y') 
Y = np.array(Y)

In [ ]:
os.chdir('../src')
print(os.getcwd())

config = tf.ConfigProto() 
config.gpu_options.per_process_gpu_memory_fraction = 0.5 
set_session(tf.Session(config=config))


In [ ]:
img_rows, img_cols = 32, 32
X = X.reshape(X.shape[0], img_cols, img_rows, 1)
Y = Y.reshape(Y.shape[0], img_cols, img_rows, 1)
#T = T.reshape(T.shape[0], img_cols, img_rows, 1)

input_img = Input(shape=(img_rows, img_cols, 1))

In [ ]:
x_c0 = Convolution2D(64, (15, 15), activation='relu', padding='same')(input_img)
x_p0 = MaxPooling2D((2, 2), padding='same')(x_c0)
x_c1 = Convolution2D(64, (11, 11), activation='relu', padding='same')(x_p0)
x_p1 = MaxPooling2D((2, 2), padding='same')(x_c1)
x_c2 = Convolution2D(128, (5, 5), activation='relu', padding='same')(x_p1)
x_p2 = MaxPooling2D((2, 2), padding='same')(x_c2)
x_c3 = Convolution2D(256, (3, 3), activation='relu', padding='same')(x_p2)
encoded = MaxPooling2D((2, 2), padding='same')(x_c3)

In [ ]:
x_2_c3_x = Conv2DTranspose(256, (3, 3), activation='relu', padding='same')(encoded)
x_2_p1 = UpSampling2D((2, 2))(x_2_c3_x)
x_2_c3 = Conv2DTranspose(256, (3, 3), activation='relu', padding='same')(x_2_p1)
m1 = Add()([x_c3, x_2_c3])

x_2_p2 = UpSampling2D((2, 2))(m1)
x_2_c2 = Conv2DTranspose(128, (5, 5), activation='relu', padding='same')(x_2_p2)

m2 = Add()([x_c2, x_2_c2])

x_2_p3 = UpSampling2D((2, 2))(m2)
x_2_c1 = Conv2DTranspose(64, (11, 11), activation='relu', padding='same')(x_2_p3)
m3 = Add()([x_c1, x_2_c1])

x_2_p4 = UpSampling2D((2, 2))(m3)
x_2_c0 = Conv2DTranspose(64, (15, 15), activation='relu', padding='same')(x_2_p4)
m4 = Add()([x_c0, x_2_c0])

decoded = Conv2DTranspose(1, (img_rows, img_cols), activation='sigmoid', padding='same')(m4)

In [ ]:
autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='mean_squared_error')
autoencoder.summary()

In [ ]:
autoencoder.fit(X, Y, validation_split=0.33, epochs=2, batch_size=50, shuffle=True, verbose=1)

h = autoencoder.history
print(h.history)
autoencoder.save('conv_ae.h5')
plot_model(autoencoder, to_file='conv_ae.png')